Importar la librerias y el archivo

In [2]:
import pandas as pd
import numpy as np

###leer archivo de excel
datos = pd.read_csv("C:/Users/Aleja/Documents/Maestria/DMEyF/DMEyF2024/Competencia_02/competencia_02_crudo.csv.gz")
datosO = datos.copy()

Agregar nueva columna de fecha

In [5]:
# Convertir la columna periodo a fecha
datos['fecha'] = pd.to_datetime(datos['foto_mes'], format='%Y%m')

# calcular los meses siguientes
datos['mes_1'] = (datos['fecha'] + pd.DateOffset(months=1)).dt.strftime('%Y%m').astype(int)
datos['mes_2'] = (datos['fecha'] + pd.DateOffset(months=2)).dt.strftime('%Y%m').astype(int)

Hacer los cruces respectivos

In [18]:
###dejar solo dos columnas del dataset2
datosO = datosO[['numero_de_cliente','foto_mes']]

###hacer los cruces
combinacion = pd.merge(datos, datosO, left_on=['numero_de_cliente','mes_1'], right_on=['numero_de_cliente','foto_mes'], how = 'left')
combinacion = pd.merge(combinacion, datosO, left_on=['numero_de_cliente','mes_2'], right_on=['numero_de_cliente','foto_mes'], how = 'left')


Crear clase ternaria

In [23]:
condiciones = [
    (combinacion['foto_mes_y']>0) & (combinacion['foto_mes']>0),
    (combinacion['foto_mes_y']>0) & (combinacion['foto_mes'].isnull()) & (combinacion['foto_mes_x'] != 202107),
    (combinacion['foto_mes_y'].isnull()) & (combinacion['foto_mes'].isnull()) & (combinacion['foto_mes_x'] != 202108),
    (combinacion['foto_mes_y'].isnull()) & (combinacion['foto_mes']>0)
]

respuesta = ['CONTINUA', 'BAJA+2', 'BAJA+1', 'CONTINUA']

combinacion['clase_ternaria'] = np.select(condiciones, respuesta, default = '')

Eliminar columnas, renombrar columna

In [24]:
###eliminar columnas
combinacion = combinacion.drop(columns=['foto_mes', 'foto_mes_y','mes_1','mes_2','fecha'])

###actualizar el nombre de una columna
combinacion = combinacion.rename(columns={'foto_mes_x': 'foto_mes'})


Revisar

In [27]:
c = combinacion.groupby(['foto_mes','clase_ternaria']).size()

Crear csv

In [28]:
combinacion.to_csv("C:/Users/Aleja/Documents/Maestria/DMEyF/DMEyF2024/Competencia_02/dataset.csv.gz")

Quitar los baja + 1

In [2]:
datos = pd.read_csv("C:/Users/Aleja/Documents/Maestria/DMEyF/DMEyF2024/Competencia_02/dataset.csv.gz")

C:\Users\Aleja\AppData\Local\Temp\ipykernel_37440\1793582643.py:1: DtypeWarning: Columns (155) have mixed types. Specify dtype option on import or set low_memory=False.
  datos = pd.read_csv("C:/Users/Aleja/Documents/Maestria/DMEyF/DMEyF2024/Competencia_02/dataset.csv.gz")


In [3]:
datos = datos[datos['clase_ternaria'] != 'BAJA+1']

In [ ]:
datos.to_csv("C:/Users/Aleja/Documents/Maestria/DMEyF/DMEyF2024/Competencia_02/dataset_SinBAJA1.csv.gz", index = False)